# Using OpenAI with ROS

<img src="img/robotignite_logo_text.png" width="700" />

## Unit 6: How to change the learning algorithm: Cube

<p style="background:green;color:white;">SUMMARY</p>

Estimated time to completion: **2 h**

In this unit, we are going to see how to set up the environment in order to be able to use the OpenAI Baselines deepq algorithm. For that, we are going to use a virtual environment. We are going to use the RoboCube environment during the Unit.

<p style="background:green;color:white;">END OF SUMMARY</p>

Now, we are going to only change the learning script from using **Qlearn** to use **DeepQ**. We won't change anything else. Here you will also see the power of environments. This allows you to change the learning algorith easily.

So, you already have a version of the CubeEnvironment to use with deepq:

<p style="background:#407EAF;color:white;">**my_start_deepq.py**</p>

In [ ]:
#!/usr/bin/env python
import os
import gym
from baselines import deepq
import rospy
import rospkg
#from openai_gazebo.cartpole_stay_up import stay_up
import my_one_disk_walk



def main():
    rospy.init_node('movingcube_onedisk_walk_gym', anonymous=True, log_level=rospy.WARN)
    
    # Set the path where learned model will be saved
    rospack = rospkg.RosPack()
    pkg_path = rospack.get_path('my_moving_cube_pkg')
    models_dir_path = os.path.join(pkg_path, "models_saved")
    if not os.path.exists(models_dir_path):
        os.makedirs(models_dir_path)
    
    out_model_file_path = os.path.join(models_dir_path, "movingcube_model.pkl") 
    
    
    max_timesteps = rospy.get_param("/moving_cube/max_timesteps")
    buffer_size = rospy.get_param("/moving_cube/buffer_size")
    # We convert to float becase if we are using Ye-X notation, it sometimes treats it like a string.
    lr = float(rospy.get_param("/moving_cube/lr"))
    
    exploration_fraction = rospy.get_param("/moving_cube/exploration_fraction")
    exploration_final_eps = rospy.get_param("/moving_cube/exploration_final_eps")
    print_freq = rospy.get_param("/moving_cube/print_freq")
    
    reward_task_learned = rospy.get_param("/moving_cube/reward_task_learned")
    
    
    def callback(lcl, _glb):
        # stop training if reward exceeds 199
        aux1 = lcl['t'] > 100
        aux2 = sum(lcl['episode_rewards'][-101:-1]) / 100
        is_solved = aux1 and aux2 >= reward_task_learned
        
        rospy.logdebug("aux1="+str(aux1))
        rospy.logdebug("aux2="+str(aux2))
        rospy.logdebug("reward_task_learned="+str(reward_task_learned))
        rospy.logdebug("IS SOLVED?="+str(is_solved))
        
        return is_solved
    
    env = gym.make("MyMovingCubeOneDiskWalkEnv-v0")
    
    
    model = deepq.models.mlp([64])
    act = deepq.learn(
        env,
        q_func=model,
        lr=lr, 
        max_timesteps=max_timesteps, 
        buffer_size=buffer_size,
        exploration_fraction=exploration_fraction,
        exploration_final_eps=exploration_final_eps,
        print_freq=print_freq, # how many apisodes until you print total rewards and info
        param_noise=False,
        callback=callback
    )
    
    env.close()
    rospy.logwarn("Saving model to movingcube_model.pkl")
    act.save(out_model_file_path)


if __name__ == '__main__':
    main()

<p style="background:#407EAF;color:white;">**END my_start_deepq.py**</p>

As you can see, we are loading all the parameters from the param server, therefore, we will have to create a new config file to be loaded when launching this deepq script. Here you have it:

<p style="background:#407EAF;color:white;">**my_one_disk_walk_openai_params_deepQ.yaml**</p>

In [ ]:
moving_cube: #namespace

    
    # DeepQ Parameters
    max_timesteps: 1000 # Maximum time steps of all the steps done throughout all the episodes
    buffer_size: 500 # size of the replay buffer
    lr: 1e-3 # learning rate for adam optimizer
    exploration_fraction: 0.1 # fraction of entire training period over which the exploration rate is annealed
    exploration_final_eps: 0.02 # final value of random action probability
    print_freq: 1 # how often (Ex: 1 means every episode, 2 every two episode we print ) to print out training progress set to None to disable printing
    
    reward_task_learned: 10000
    
    # Learning General Parameters
    n_actions: 5 # We have 3 actions
    n_observations: 6 # We have 6 different observations

    speed_step: 1.0 # Time to wait in the reset phases

    init_roll_vel: 0.0 # Initial speed of the Roll Disk

    roll_speed_fixed_value: 100.0 # Speed at which it will move forwards or backwards
    roll_speed_increment_value: 10.0 # Increment that could be done in each step

    max_distance: 2.0 # Maximum distance allowed for the RobotCube
    max_pitch_angle: 0.2 # Maximum Angle radians in Pitch that we allow before terminating episode
    max_yaw_angle: 0.1 # Maximum yaw angle deviation, after that it starts getting negative rewards

    max_y_linear_speed: 100 # Free fall get to around 30, so we triple it 

    init_cube_pose:
      x: 0.0
      y: 0.0
      z: 0.0

    end_episode_points: 1000 # Points given when ending an episode

    move_distance_reward_weight: 1000.0 # Multiplier for the moved distance reward, Ex: inc_d = 0.1 --> 100points
    y_linear_speed_reward_weight: 1000.0 # Multiplier for moving fast in the y Axis
    y_axis_angle_reward_weight: 1000.0 # Multiplier of angle of yaw, to keep it straight

    


<p style="background:#407EAF;color:white;">**END my_one_disk_walk_openai_params_deepQ.yaml**</p>

Just comment on the **deepq** related parameters; the other ones are exactly the same as before:

In [ ]:
# DeepQ Parameters
max_timesteps: 1000 # Maximum time steps of all the steps done throughout all the episodes
buffer_size: 500 # size of the replay buffer
lr: 1e-3 # learning rate for adam optimizer
exploration_fraction: 0.1 # fraction of entire training period over which the exploration rate is annealed
exploration_final_eps: 0.02 # final value of random action probability
print_freq: 1 # how often (Ex: 1 means every episode, 2 every two episode we print ) to print out training progress set to None to disable printing

reward_task_learned: 10000

**max_timesteps**: This parameter allows you to set the maximum **TOTAL** steps throughout the entire learning process. It doesn't matter if you do 1000 steps in 1000 episodes, or in one episode.

**print_freq**: We define how many episodes until we print the average reward and other learning info.

**reward_task_learned**: When the sum of all the rewards in that episode reach this value, it will consider the robot to have learned the task.

But this model has many more options, so please feel free to experiment. Here you have all the parameter options from the deepQ class:

And now we have to follow the same steps you took for the **CartPole** to setup the **Baselines,** if you haven't done it yet.

## 1. Set Up the Baselines for DeepQ

#### Creation of Python3_ws

The first thing you have to do is create a **Python 3** workspace. This is due to the fact that ROS doesn't directly support **Python 3**. This means that if you launch programs for ROS with libraries for Python 3, it will most likely give an error.

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1</p>
</th>
</tr>
</table>

In [ ]:
cd
source /opt/ros/kinetic/setup.bash
mkdir -p ~/python3_ws/src
cd ~/python3_ws/
catkin_make

#### Prepare the Baselines Git Environment

Now you will have to move your code to the **python3_ws**.

Once you have it, let's downlad the baselines git and setup the **Virtual Environment,** which uses, among other things, Python 3.

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1</p>
</th>
</tr>
</table>

We activate the Virtual Environment:

And we install all the dependencies needed, within the safe confines of our Virtual Environment:

Great! So now everything is ready to start using deepq!

#### Compile ROS TF for Python 3

Because we are using these ROS libraries in our code, we will need to compile them in our **Virtual Environment**.

In [ ]:
cd ~/python3_ws/src;cd baselines
source venv/bin/activate

pip install catkin_pkg pyyaml empy rospkg numpy
cd ~/python3_ws/src
git clone https://github.com/ros/geometry
git clone https://github.com/ros/geometry2
cd ~/python3_ws
rm -rf build devel
catkin_make --force-cmake
source devel/setup.bash

To test that you are using **Python 3** and that everythng works, execute the following in an **ipython** interpreter:

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1</p>
</th>
</tr>
</table>

To test that the tf2 works with Python 3:

In [ ]:
(venv) user:~/catkin_ws$ python
Python 3.5.2 (default, Nov 23 2017, 16:37:01)
[GCC 5.4.0 20160609] on linux
Type "help", "copyright", "credits" or "license" for more information.
>>> import tf
>>> tf.__file__
'/home/user/catkin_ws/devel/lib/python3/dist-packages/tf/__init__.py'
>>> import tf2_py
>>> tf2_py.__file__
'/home/user/catkin_ws/devel/lib/python3/dist-packages/tf2_py/__init__.py'

It should give you an output like this one, using Python 3.

#### How to get inside the VENV every time you open a new shell

You will have to do this very often, so here are the commands:

In [ ]:
cd ~/python3_ws/src/baselines
source venv/bin/activate
cd ~/python3_ws
source devel/setup.bash

## 2. Start the Learning script using DeepQ for the MyMovingCubeOneDiskWalkEnv

We just have to create a new launch file that launches the deepq Python script, instead of the Qlearn learning script:

<p style="background:#407EAF;color:white;">**my_start_training_deepq_version.launch**</p>

In [ ]:
<launch>
    <rosparam command="load" file="$(find my_moving_cube_pkg)/config/my_one_disk_walk_openai_params_deepQ.yaml" />
    <!-- Launch the training system -->
    <node pkg="my_moving_cube_pkg" name="movingcube_gym" type="my_start_deepq.py" output="screen"/>
</launch>

As you can see, we didn't change much.

<p style="background:#407EAF;color:white;">**END my_start_training_deepq_version.launch**</p>

To start the learning, get inside the venv and launch the deepq launch.

In [ ]:
cd ~/python3_ws/src/baselines
source venv/bin/activate
cd ~/python3_ws
source devel/setup.bash
roslaunch my_moving_cube_pkg my_start_training_deepq_version.launch

You should get something like this:

<img src="img/cube_deepQ.gif" />

When the total steps done reaches the maximum, you have established or reached the reward for the task to be considered learned. Now, the system will automatically create a **pickle** file for you, with the model of the learned task inside.

<p style="background:#FF5E00;color:white;">**EXTRA exercise U3.1**</p>

Change the parameter so that:
* It spends more time learning
* It stops learning when it reaches a lower reward average
* It prints the learning information less frequently.

<p style="background:#FF5E00;color:white;">**END EXTRA exercise U3.1**</p>

## And that's it! You now know how to reuse Task Environments for different learning algorithms.